In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [ ]:
url = 'https://www.timeshighereducation.com/world-university-rankings/latest/world-ranking'

In [ ]:
driver = webdriver.Chrome()
driver.maximize_window()
driver.get(url)

# Закрыть окно куки
try:
    accept_btn = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Allow all')]"))
    )
    driver.execute_script("arguments[0].click();", accept_btn)
    time.sleep(2)
except:
    pass

# Подождать таблицу
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "table tbody tr"))
)
try:
    key_stats_tab = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Key statistics')]"))
    )
    key_stats_tab.click()
    time.sleep(2)  # подождать загрузку контента
except Exception as e:
    print(f"Не удалось переключиться на Key statistics: {e}")
    
# Собирать ВСЕ строки
all_data = []
seen_hrefs = set()  # отслеживать по ссылке, чтобы избежать дублей

last_new_count = 0
no_change_count = 0
scroll_index = 0

for attempt in range(2000):
    rows = driver.find_elements(By.CSS_SELECTOR, "table tbody tr")
    
    for row in rows:
        cells = row.find_elements(By.TAG_NAME, "td")
        if len(cells) == 0:
            continue
        
        # Извлечь текст из ячеек
        row_data = [cell.text.strip() for cell in cells]
        
        if not any(row_data):
            continue
        
        # Извлечь ссылку на страницу вуза 
        try:
            link_element = cells[1].find_element(By.TAG_NAME, "a")
            uni_url = link_element.get_attribute("href")
        except:
            uni_url = ""
        
        # Добавлять только если ссылка уникальная
        if uni_url and uni_url not in seen_hrefs:
            seen_hrefs.add(uni_url)
            row_data.append(uni_url)  # добавить ссылку последней колонкой
            all_data.append(row_data)
    
    current_total = len(all_data)
    
    if current_total > last_new_count:
        last_new_count = current_total
        no_change_count = 0
    else:
        no_change_count += 1
        if no_change_count >= 25:
            print("Новых строк не появляется, завершаем")
            break
    
    # Скроллить постепенно
    if len(rows) > 10:
        target_index = min(len(rows) - 5, scroll_index)
        if target_index < len(rows):
            driver.execute_script("""
                arguments[0].scrollIntoView({block: 'center', inline: 'nearest'});
            """, rows[target_index])
        scroll_index += 3
    
    time.sleep(1)
    
driver.quit()
print(f"Всего университетов: {len(all_data)}")
